# Setting Vertex AI Agent API for MongoDB Vertex AI Extensions.


 ## Vertex AI Agent API

 The [Vertex AI Agent API](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-api/overview) is an API for creating and managing generative AI systems called "agents" that can  reason, plan, and act to perform specific tasks.

## Using This Notebook

Colab is recommended for running this notebook, but it can run in any iPython environment where you can connect to Google Cloud, install pip packages, etc.

If you're running outside of Colab, depending on your environment you may need to install pip packages that are included in the Colab environment by default but are not part of the Python Standard Library--try pipping the library name of any imports that fail.

This tutorial uses the following Google Cloud services and resources:

* [Vertex AI Agent API](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-api/overview)

This notebook has been tested in the following environment:

* Python version = 3.10.12
* [google-cloud-aiplatform](https://pypi.org/project/google-cloud-aiplatform/) version = 1.70

## Useful Tips

1. This notebook uses Generative AI cababilities. Re-running a cell that uses Generative AI capabilities may produce similar but not identical results.
2. Because of #1, it is possible that an output produces errors. If that happens re-run the cell that produced the error. The re-run will likely be bug free.
3. The use of Generative AI capabilities is subject to service quotas. Running the notebook using "Run All" may exceed your queries per minute (QPM) limitations. Run the notebook manually and if you get a quota error pause for up to 1 minute before retrying that cell. The Vertex AI Agent API defaults to Gemini on the backend and is subject to the Gemini quotas, [view your Gemini quotas here](https://console.cloud.google.com/iam-admin/quotas?pageState=(%22allQuotasTable%22:(%22f%22:%22%255B%257B_22k_22_3A_22_22_2C_22t_22_3A10_2C_22v_22_3A_22_5C_22base_model_5C_22_22%257D_2C%257B_22k_22_3A_22_22_2C_22t_22_3A10_2C_22v_22_3A_22_5C_22gemini_5C_22_22%257D%255D%22%29%29&e=13802955&mods=logs_tg_staging).


# Getting Started

The following steps are necessary to run this notebook, no matter what notebook environment you're using.

If you're entirely new to Google Cloud, [get started here](https://cloud.google.com/docs/get-started).

## Google Cloud Project Setup

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

## Google Cloud Permissions
Make sure you have been [granted the following roles](https://cloud.google.com/iam/docs/granting-changing-revoking-access) for the GCP project you'll access from this notebook:
* [`roles/aiplatform.user`](https://cloud.google.com/vertex-ai/docs/general/access-control#aiplatform.user)

## Authenticate

If you're using Colab, run the code in the next cell. Follow the popups and authenticate with an account that has access to your Google Cloud [project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects).

If you're running this notebook somewhere besides Colab, make sure your environment has the right Google Cloud access. If that's a new concept to you, consider looking into [Application Default Credentials for your local environment](https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev) and [initializing the Google Cloud CLI](https://cloud.google.com/docs/authentication/gcloud). In many cases, running `gcloud auth application-default login` and/or `gcloud auth login` in a shell on the machine running the notebook kernel is sufficient.

More authentication options are discussed [here](https://cloud.google.com/docs/authentication).

In [10]:
# Colab authentication.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
    print("Authenticated")

Authenticated


## Install the Google Cloud Vertex AI Python SDK

Install the Google Cloud Vertex AI Python SDK, and if you already have the Google Cloud Vertex AI Python SDK installed, upgrade to the latest version.

In [11]:
from google.colab import auth
auth.authenticate_user()
# Outside of colab, gcloud auth login

In [ ]:
# Install gcloud
!pip install --quiet google-cloud

WHL_FILEPATH = " gs://vertex_sdk_private_releases/agents_v2/google_cloud_aiplatform-1.70.dev20241008+vertex.agents.v2-py2.py3-none-any.whl"
!gsutil cp {WHL_FILEPATH} .

# Install the private SDK
!pip install --quiet {WHL_FILEPATH.split("/")[-1]} "numpy<2.0.0" --force-reinstall

### Restart Runtime

You may need to restart your notebook runtime to use the Vertex AI SDK. You can do this by running the cell below, which restarts the current kernel.

You may see the restart reported as a crash, but it is working as-intended -- you are merely restarting the runtime.

The restart might take a minute or longer. After its restarted, continue to the next step.

In [13]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


# Initialize the Google Cloud Vertex AI Python SDK

Start here if your Notebook kernel restarts (but isn't deleted), though if it's been a few hours you may need to run the Authentication steps above again.

To initialize the SDK, you need to set your Google Cloud project ID and region.

If you don't know your project  ID, try the [Google Cloud CLI](https://cloud.google.com/sdk) commands [`gcloud config list`](https://cloud.google.com/sdk/gcloud/reference/config/list) or [`gcloud projects list`](https://cloud.google.com/sdk/gcloud/reference/projects/list). See the support page [Locate the project ID](https://support.google.com/googleapi/answer/7014113) for more information.


### Set Your Project ID

In [1]:
PROJECT_ID = ""  # @param {type:"string"}

# Set the project for CLI commands.
# !gcloud config set project {PROJECT_ID}

### Set the Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [2]:
REGION = "us-central1"  # @param {type: "string"}

### Import and Initialize the Vertex AI Python SDK

In [3]:
import vertexai
from google.cloud.aiplatform.private_preview.vertex_agents_v2 import agents,sessions
from vertexai.generative_models import FunctionDeclaration, Part
from vertexai.preview.extensions import Extension

vertexai.init(project=PROJECT_ID, location=REGION)

# Get Vertex AI Extension for MongoDB


In [ ]:
from google.cloud import aiplatform
from vertexai.preview import extensions

EXTENSIONS_ID="projects/787220387490/locations/us-central1/extensions/8830380192808566784" # @param {type:"string"}

mdb_extension = extensions.Extension(EXTENSIONS_ID)
mdb_extension

# Create an Agent

An AI agent reasons, plans, and takes actions. The agent takes actions via access to **tools**, deciding how and when to invoke a tool. The agent also manages orchestration, creating a plan for answering a user query and adapting to responses that aren't quite correct.

Agent **tools** are either [Vertex AI Extensions](https://cloud.google.com/vertex-ai/generative-ai/docs/extensions/overview) or functions (set up using [function calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling)).

This Notebook covers steps to setup tools using Vertex AI Extensions. Follow the [tutorial](https://medium.com/google-cloud/adding-context-to-rag-with-gemini-function-calling-d6e78e705524) to learn how to set up Function calling on MongoDB Atlas.

To create a simple agent, An agent requires two things: a display name and a set of instructions. The instructions should be detailed, making it clear exactly how the agent should behave. We are considering a chatbot agent that can query multiple MongoDB collections.

Note: Update the Cluster name in below code snippet with the Clsuter you have setup on MongoDB



In [ ]:
DISPLAY_NAME = "MongoDB Service Agent"
INSTRUCTIONS = """
You are a chatbot that makes api call to mongodb cluster <Cluster Name>,
database 'sample_mflix' and collection 'movies' it also connects to database
'sample_restaurants' and collection 'restaurants' to do all crud operations
"""

mdb_agent = agents.create(
    display_name=DISPLAY_NAME,
    instruction=INSTRUCTIONS,
    model="gemini-1.5-pro",
    tools=[mdb_extension]
    )

## Interact with an Agent

To interact with the agent, create a session and then use `run` to send your inputs.

In [ ]:
mdb_session = sessions.create(display_name="mongodb-session")

Keep using `.create_run` to have a conversation with the agent.

In [ ]:
run = mdb_session.create_run(agent=mdb_agent, content="get me movie name staring Will Smilth")
run.steps

# Handle Function Calls

We will create a function that returns the queried data from MongoDB Atlas by the Agent.

In [51]:
def mdb_service_agent(input_text):
  response = mdb_session.create_run(agent=mdb_agent,content=input_text)
  response
  return response.steps[-1].content.parts[0].text

# Test the MongoDB Agent Service

In [52]:
mdb_service_agent("get me the borough of the resturant named Riviera Caterer")

'The borough of the restaurant named Riviera Caterer is Brooklyn. \n'

In [55]:
mdb_service_agent("get me movie name staring will smilth")

'Here are up to 10 movie names starring Will Smith: Made in America, Six Degrees of Separation, Bad Boys, Independence Day, Men in Black, Enemy of the State, Wild Wild West, Men in Black II, The Legend of Bagger Vance, and Bad Boys II. \n\n\n\n'